# **STEP 1: Load & Understand the Data**

In [3]:
import pandas as pd
import numpy as np

# Set display options
pd.set_option("display.max_columns", None)

# Load datasets
customers = pd.read_csv("/content/customers.csv")
invoices = pd.read_csv("/content/invoices.csv")
payments = pd.read_csv("/content/payments.csv")
purchase_orders = pd.read_csv("/content/purchase_orders.csv")
interactions = pd.read_csv("/content/customer_interactions.csv")

# Quick overview
datasets = {
    "Customers": customers,
    "Invoices": invoices,
    "Payments": payments,
    "Purchase Orders": purchase_orders,
    "Interactions": interactions
}

for name, df in datasets.items():
    print(f"\n{name}")
    print("Shape:", df.shape)
    print(df.head())



Customers
Shape: (10000, 7)
  customer_id customer_name   industry                  email       phone  \
0      C00001    Customer_1  Logistics  customer1@company.com  9573348541   
1      C00002    Customer_2   Services  customer2@company.com  4518824419   
2      C00003    Customer_3     Retail  customer3@company.com  3122039319   
3      C00004    Customer_4   Services  customer4@company.com  2551970603   
4      C00005    Customer_5   Services  customer5@company.com  2112349238   

            city account_status  
0  Washington DC         Active  
1  Washington DC        On Hold  
2  Washington DC         Active  
3      Rockville         Active  
4       Columbia         Active  

Invoices
Shape: (20000, 6)
  invoice_id customer_id invoice_date  invoice_amount    due_date  \
0  INV100000      C09769   2024-04-21           13110  2024-05-21   
1  INV100001      C06097   2024-03-20           15641  2024-04-19   
2  INV100002      C07356   2024-02-17           20188  2024-03-18   


Basic Data Understanding

In [4]:
# Check missing values
for name, df in datasets.items():
    print(f"\n{name} - Missing Values")
    print(df.isnull().sum())

# Check data types
for name, df in datasets.items():
    print(f"\n{name} - Data Types")
    print(df.dtypes)



Customers - Missing Values
customer_id       0
customer_name     0
industry          0
email             0
phone             0
city              0
account_status    0
dtype: int64

Invoices - Missing Values
invoice_id        0
customer_id       0
invoice_date      0
invoice_amount    0
due_date          0
invoice_status    0
dtype: int64

Payments - Missing Values
invoice_id        0
payment_id        0
payment_date      0
payment_amount    0
payment_method    0
dtype: int64

Purchase Orders - Missing Values
po_id              0
customer_id        0
po_date            0
po_amount          0
approval_status    0
dtype: int64

Interactions - Missing Values
interaction_id      0
customer_id         0
date                0
interaction_type    0
issue_category      0
notes               0
dtype: int64

Customers - Data Types
customer_id       object
customer_name     object
industry          object
email             object
phone              int64
city              object
account_status   

# **STEP 2: Data Cleaning & validation**

1. Remove Duplicates

In [5]:
customers.drop_duplicates(subset="customer_id", inplace=True)
invoices.drop_duplicates(subset="invoice_id", inplace=True)
payments.drop_duplicates(subset="payment_id", inplace=True)
purchase_orders.drop_duplicates(subset="po_id", inplace=True)
interactions.drop_duplicates(subset="interaction_id", inplace=True)


2. Handle Missing or Invalid Values


In [6]:
# Fill missing contact details - correct approach
customers["email"] = customers["email"].fillna("missing@company.com")
customers["phone"] = customers["phone"].fillna(0)

# Remove invoices with invalid amounts
invoices = invoices[invoices["invoice_amount"] > 0]

# Remove payments with invalid amounts
payments = payments[payments["payment_amount"] > 0]

3. Convert Date Columns to Datetime

In [7]:
date_columns = {
    "invoices": ["invoice_date", "due_date"],
    "payments": ["payment_date"],
    "purchase_orders": ["po_date"],
    "interactions": ["date"]
}

for df_name, cols in date_columns.items():
    df = globals()[df_name]
    for col in cols:
        df[col] = pd.to_datetime(df[col])


4. Validation Checks(VERY IMPORTANT)



*   Invoice → Customer Validation



In [8]:
invalid_invoices = invoices[~invoices["customer_id"].isin(customers["customer_id"])]
print("Invoices without valid customers:", len(invalid_invoices))


Invoices without valid customers: 0




*  Payment Amount ≤ Invoice Amount



In [9]:
invoice_amount_map = invoices.set_index("invoice_id")["invoice_amount"]

payments["invoice_amount"] = payments["invoice_id"].map(invoice_amount_map)

invalid_payments = payments[payments["payment_amount"] > payments["invoice_amount"]]
print("Invalid payments:", len(invalid_payments))


Invalid payments: 0


5. Create a Clean Master AR Dataset

In [10]:
# Merge invoices with customers
ar_master = invoices.merge(customers, on="customer_id", how="left")

# Aggregate payments per invoice
payment_summary = payments.groupby("invoice_id", as_index=False)["payment_amount"].sum()

# Merge payment totals
ar_master = ar_master.merge(payment_summary, on="invoice_id", how="left")

# Fix: assign the result back instead of using inplace=True
ar_master["payment_amount"] = ar_master["payment_amount"].fillna(0)

# Calculate balance
ar_master["outstanding_balance"] = (
    ar_master["invoice_amount"] - ar_master["payment_amount"]
)

In [11]:
ar_master.sample(10)

,invoice_id,customer_id,invoice_date,invoice_amount,due_date,invoice_status,customer_name,industry,email,phone,city,account_status,payment_amount,outstanding_balance
4089,INV104089,C07194,2024-08-03,7838,2024-09-02,Open,Customer_7194,Logistics,customer7194@company.com,9090017725,Columbia,On Hold,7838.0,0.0
19742,INV119742,C09199,2024-03-25,12114,2024-05-09,Paid,Customer_9199,Retail,customer9199@company.com,4843309419,Fairfax,Active,12114.0,0.0
14166,INV114166,C04048,2024-05-10,17247,2024-06-09,Open,Customer_4048,Logistics,customer4048@company.com,5819248735,Columbia,Active,17247.0,0.0
1549,INV101549,C02896,2024-09-21,4244,2024-11-05,Overdue,Customer_2896,Healthcare,customer2896@company.com,3080609350,Columbia,On Hold,0.0,4244.0
13178,INV113178,C04784,2024-03-07,12925,2024-04-21,Paid,Customer_4784,Manufacturing,customer4784@company.com,7825545862,Columbia,Active,0.0,12925.0
3288,INV103288,C04198,2024-01-28,24100,2024-03-28,Open,Customer_4198,Retail,customer4198@company.com,4009682341,Rockville,Active,24100.0,0.0
5128,INV105128,C02525,2024-05-04,14165,2024-06-18,Paid,Customer_2525,Logistics,customer2525@company.com,9353491055,Baltimore,Active,14165.0,0.0
16472,INV116472,C08328,2024-01-10,3764,2024-03-10,Paid,Customer_8328,Logistics,customer8328@company.com,7025461836,Rockville,Active,3764.0,0.0
17050,INV117050,C08242,2024-09-24,2932,2024-11-08,Open,Customer_8242,Logistics,customer8242@company.com,4176161909,Washington DC,Active,0.0,2932.0
11,INV100011,C01470,2024-08-21,714,2024-10-05,Paid,Customer_1470,Retail,customer1470@company.com,6963587251,Washington DC,Active,0.0,714.0


6. Invoice Aging

In [12]:
as_of_date = invoices["invoice_date"].max()

ar_master["days_past_due"] = (as_of_date - ar_master["due_date"]).dt.days
ar_master["days_past_due"] = ar_master["days_past_due"].clip(lower=0)

ar_master["aging_bucket"] = pd.cut(
    ar_master["days_past_due"],
    bins=[-1, 0, 30, 60, 90, 9999],
    labels=["Current", "0-30", "31-60", "61-90", "90+"]
)

6. Final Validation Summary

In [13]:
print("Total Invoices:", len(ar_master))
print("Total Outstanding Balance:", ar_master["outstanding_balance"].sum())
print("\nAging Distribution:")
print(ar_master["aging_bucket"].value_counts())


Total Invoices: 20000
Total Outstanding Balance: 83429432.5

Aging Distribution:
aging_bucket
90+        12599
Current     2560
61-90       1650
31-60       1623
0-30        1568
Name: count, dtype: int64


# **STEP 3: Analysis & Key Metrics**



*  AR Aging Calculation



In [14]:
# Use latest invoice date as reporting date
as_of_date = invoices["invoice_date"].max()

ar_master["days_past_due"] = (as_of_date - ar_master["due_date"]).dt.days
ar_master["days_past_due"] = ar_master["days_past_due"].clip(lower=0)

ar_master["aging_bucket"] = pd.cut(
    ar_master["days_past_due"],
    bins=[-1, 0, 30, 60, 90, 9999],
    labels=["Current", "0-30", "31-60", "61-90", "90+"]
)




*   Payment cycle analysis



In [15]:
# Merge earliest payment date per invoice
payment_dates = (
    payments.groupby("invoice_id")["payment_date"]
    .min()
    .reset_index()
)

ar_master = ar_master.merge(payment_dates, on="invoice_id", how="left")

# Payment cycle (days)
ar_master["payment_cycle_days"] = (
    ar_master["payment_date"] - ar_master["invoice_date"]
).dt.days




*   Accounts Receivable KPIs




In [16]:
# Total AR
total_ar = ar_master["outstanding_balance"].sum()

# Overdue AR
overdue_ar = ar_master.loc[
    ar_master["aging_bucket"] != "Current", "outstanding_balance"
].sum()

# Percentage overdue
overdue_pct = overdue_ar / total_ar * 100

total_ar, overdue_ar, overdue_pct


(np.float64(83429432.5), np.float64(72787204.0), np.float64(87.24403585029779))



*   Average Days to Collect


In [17]:
avg_days_to_collect = (
    ar_master.loc[
        ar_master["invoice_status"] == "Paid",
        "payment_cycle_days"
    ]
    .mean()
)

avg_days_to_collect


np.float64(56.080900243309)





*  High-Risk Customers



In [18]:
high_risk_customers = (
    ar_master[ar_master["aging_bucket"].isin(["61-90", "90+"])]
    .groupby("customer_id", as_index=False)
    .agg(
        total_outstanding=("outstanding_balance", "sum"),
        invoice_count=("invoice_id", "count")
    )
    .sort_values("total_outstanding", ascending=False)
)

high_risk_customers.head(10)


,customer_id,total_outstanding,invoice_count
4080,C05371,67851.0,5
2040,C02662,66654.0,4
6514,C08522,66036.0,5
4807,C06311,63350.0,4
6110,C07997,62941.5,4
6963,C09124,60467.5,4
1017,C01341,55856.5,5
7024,C09206,54761.0,3
5736,C07519,53473.0,3
4687,C06157,53435.0,5


# **STEP 4: Customer Interaction Analysis**



*  Link interactions to AR risk



In [19]:
interaction_summary = (
    interactions.groupby(["customer_id", "issue_category"])
    .size()
    .reset_index(name="interaction_count")
)

interaction_summary.head()


,customer_id,issue_category,interaction_count
0,C00001,Billing Question,1
1,C00001,Late Payment,1
2,C00002,Billing Question,1
3,C00003,Service Issue,2
4,C00004,Late Payment,3




*  Customers with payment issues + overdue AR



In [20]:
late_payment_customers = interactions[
    interactions["issue_category"] == "Late Payment"
]["customer_id"].unique()

late_payment_risk = high_risk_customers[
    high_risk_customers["customer_id"].isin(late_payment_customers)
]

late_payment_risk.head()


,customer_id,total_outstanding,invoice_count
6514,C08522,66036.0,5
4807,C06311,63350.0,4
1017,C01341,55856.5,5
1107,C01462,52666.0,4
218,C00280,52297.0,3


# **STEP 5: Operational Process Review (PO & Invoice Bottlenecks)**



*  Purchase Order Approval Delays



In [21]:
po_status_summary = (
    purchase_orders.groupby("approval_status")
    .size()
    .reset_index(name="count")
)

po_status_summary


,approval_status,count
0,Approved,8391
1,Delayed,1181
2,Pending,2428




*  Delayed POs vs Overdue Invoices



In [22]:
delayed_po_customers = purchase_orders[
    purchase_orders["approval_status"] == "Delayed"
]["customer_id"].unique()

delayed_po_ar = ar_master[
    ar_master["customer_id"].isin(delayed_po_customers)
]

delayed_po_ar["outstanding_balance"].sum()


np.float64(9113169.0)

# **STEP 6: Reporting & Power BI Dashboarding**

**1. Weekly AR Report**

In [ ]:
!pip install xlsxwriter

In [25]:
# ============================================================================
# WEEKLY AR REPORT
# ============================================================================
import pandas as pd
import numpy as np
from datetime import datetime

# Create Excel writer
writer = pd.ExcelWriter('/content/Weekly_AR_Report.xlsx', engine='xlsxwriter')
workbook = writer.book

# Define formats
header_format = workbook.add_format({
    'bold': True, 'bg_color': '#4472C4', 'font_color': 'white',
    'border': 1, 'align': 'center'
})
title_format = workbook.add_format({'bold': True, 'font_size': 16})
subtitle_format = workbook.add_format({'italic': True, 'font_size': 10})
currency_format = workbook.add_format({'num_format': '$#,##0.00'})
percent_format = workbook.add_format({'num_format': '0.00%'})
number_format = workbook.add_format({'num_format': '#,##0'})
date_format = workbook.add_format({'num_format': 'yyyy-mm-dd'})

# -------------------------
# SHEET 1: Executive Summary
# -------------------------
summary_data = {
    'Metric': [
        'Total AR Balance',
        'Total Overdue AR',
        'Percentage Overdue',
        'Average Days to Collect',
        'Total Open Invoices',
        'Total Paid Invoices',
        'High-Risk Customers (90+ days)',
        'Customers On Hold'
    ],
    'Value': [
        ar_master['outstanding_balance'].sum(),
        ar_master[ar_master['aging_bucket'] != 'Current']['outstanding_balance'].sum(),
        (ar_master[ar_master['aging_bucket'] != 'Current']['outstanding_balance'].sum() /
         ar_master['outstanding_balance'].sum()),
        ar_master[ar_master['invoice_status'] == 'Paid']['payment_cycle_days'].mean(),
        len(ar_master[ar_master['invoice_status'] == 'Open']),
        len(ar_master[ar_master['invoice_status'] == 'Paid']),
        len(ar_master[ar_master['aging_bucket'] == '90+']['customer_id'].unique()),
        len(customers[customers['account_status'] == 'On Hold'])
    ]
}

df_summary = pd.DataFrame(summary_data)
df_summary.to_excel(writer, sheet_name='Executive Summary', index=False, startrow=3)

worksheet = writer.sheets['Executive Summary']
worksheet.write('A1', 'WEEKLY ACCOUNTS RECEIVABLE REPORT', title_format)
worksheet.write('A2', f'Report Date: {datetime.now().strftime("%Y-%m-%d")}', subtitle_format)

for col_num, value in enumerate(df_summary.columns.values):
    worksheet.write(3, col_num, value, header_format)

worksheet.set_column('A:A', 35)
worksheet.set_column('B:B', 20)

# -------------------------
# SHEET 2: Aging Analysis
# -------------------------
aging_analysis = ar_master.groupby('aging_bucket').agg({
    'outstanding_balance': 'sum',
    'invoice_id': 'count'
}).reset_index()
aging_analysis.columns = ['Aging Bucket', 'Total Balance', 'Invoice Count']

total_balance = aging_analysis['Total Balance'].sum()
aging_analysis['% of Total'] = aging_analysis['Total Balance'] / total_balance

bucket_order = ['Current', '0-30', '31-60', '61-90', '90+']
aging_analysis['order'] = aging_analysis['Aging Bucket'].map({b: i for i, b in enumerate(bucket_order)})
aging_analysis = aging_analysis.sort_values('order').drop('order', axis=1)

aging_analysis.to_excel(writer, sheet_name='Aging Analysis', index=False, startrow=2)

worksheet = writer.sheets['Aging Analysis']
worksheet.write('A1', 'AR AGING BREAKDOWN', title_format)

for col_num, value in enumerate(aging_analysis.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:A', 15)
worksheet.set_column('B:B', 18)
worksheet.set_column('C:C', 15)
worksheet.set_column('D:D', 12)

# -------------------------
# SHEET 3: Top 20 High-Risk Customers
# -------------------------
high_risk = ar_master[ar_master['aging_bucket'].isin(['61-90', '90+'])].groupby(
    ['customer_id', 'customer_name', 'industry', 'city']
).agg({
    'outstanding_balance': 'sum',
    'invoice_id': 'count',
    'days_past_due': 'max'
}).reset_index()

high_risk.columns = ['Customer ID', 'Customer Name', 'Industry', 'City',
                    'Outstanding Balance', 'Invoice Count', 'Max Days Past Due']
high_risk = high_risk.sort_values('Outstanding Balance', ascending=False).head(20)

high_risk.to_excel(writer, sheet_name='High-Risk Customers', index=False, startrow=2)

worksheet = writer.sheets['High-Risk Customers']
worksheet.write('A1', 'TOP 20 HIGH-RISK CUSTOMERS (60+ Days Overdue)', title_format)

for col_num, value in enumerate(high_risk.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:B', 15)
worksheet.set_column('C:D', 15)
worksheet.set_column('E:E', 20)
worksheet.set_column('F:G', 15)

# -------------------------
# SHEET 4: Industry Breakdown
# -------------------------
industry_analysis = ar_master.groupby('industry').agg({
    'outstanding_balance': 'sum',
    'invoice_id': 'count',
    'payment_cycle_days': 'mean'
}).reset_index()

industry_analysis.columns = ['Industry', 'Total Outstanding',
                             'Invoice Count', 'Avg Payment Cycle (Days)']
industry_analysis = industry_analysis.sort_values('Total Outstanding', ascending=False)

industry_analysis.to_excel(writer, sheet_name='Industry Analysis', index=False, startrow=2)

worksheet = writer.sheets['Industry Analysis']
worksheet.write('A1', 'OUTSTANDING AR BY INDUSTRY', title_format)

for col_num, value in enumerate(industry_analysis.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:A', 20)
worksheet.set_column('B:B', 20)
worksheet.set_column('C:C', 15)
worksheet.set_column('D:D', 25)

# -------------------------
# SHEET 5: City Analysis
# -------------------------
city_analysis = ar_master.groupby('city').agg({
    'outstanding_balance': 'sum',
    'invoice_id': 'count',
    'customer_id': 'nunique'
}).reset_index()

city_analysis.columns = ['City', 'Total Outstanding', 'Invoice Count', 'Customer Count']
city_analysis = city_analysis.sort_values('Total Outstanding', ascending=False)

city_analysis.to_excel(writer, sheet_name='City Analysis', index=False, startrow=2)

worksheet = writer.sheets['City Analysis']
worksheet.write('A1', 'OUTSTANDING AR BY CITY', title_format)

for col_num, value in enumerate(city_analysis.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:A', 20)
worksheet.set_column('B:B', 20)
worksheet.set_column('C:D', 15)

# Save and close
writer.close()

print("✓ Weekly AR Report generated: /content/Weekly_AR_Report.xlsx")
print(f"  - Executive Summary")
print(f"  - Aging Analysis")
print(f"  - Top 20 High-Risk Customers")
print(f"  - Industry Analysis")
print(f"  - City Analysis")

✓ Weekly AR Report generated: /content/Weekly_AR_Report.xlsx
  - Executive Summary
  - Aging Analysis
  - Top 20 High-Risk Customers
  - Industry Analysis
  - City Analysis


/tmp/ipython-input-233079087.py:67: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  aging_analysis = ar_master.groupby('aging_bucket').agg({


**2. Monthly Operations Report**

In [26]:
# ============================================================================
# MONTHLY OPERATIONS REPORT
# ============================================================================
import pandas as pd
import numpy as np
from datetime import datetime

# Create Excel writer
writer = pd.ExcelWriter('/content/Monthly_Operations_Report.xlsx', engine='xlsxwriter')
workbook = writer.book

# Define formats
header_format = workbook.add_format({
    'bold': True, 'bg_color': '#70AD47', 'font_color': 'white',
    'border': 1, 'align': 'center'
})
title_format = workbook.add_format({'bold': True, 'font_size': 16})
subtitle_format = workbook.add_format({'italic': True, 'font_size': 10})
currency_format = workbook.add_format({'num_format': '$#,##0.00'})
percent_format = workbook.add_format({'num_format': '0.00%'})
number_format = workbook.add_format({'num_format': '#,##0'})

# -------------------------
# SHEET 1: Operations Summary
# -------------------------
ops_summary = {
    'Category': ['Invoices', 'Invoices', 'Invoices', 'Payments', 'Payments',
                 'Purchase Orders', 'Purchase Orders', 'Purchase Orders',
                 'Customer Interactions', 'Customer Interactions'],
    'Metric': [
        'Total Invoices',
        'Open Invoices',
        'Overdue Invoices',
        'Total Payments Received',
        'Average Payment Amount',
        'Total Purchase Orders',
        'Approved POs',
        'Delayed POs',
        'Total Interactions',
        'Late Payment Issues'
    ],
    'Value': [
        len(invoices),
        len(invoices[invoices['invoice_status'] == 'Open']),
        len(invoices[invoices['invoice_status'] == 'Overdue']),
        payments['payment_amount'].sum(),
        payments['payment_amount'].mean(),
        len(purchase_orders),
        len(purchase_orders[purchase_orders['approval_status'] == 'Approved']),
        len(purchase_orders[purchase_orders['approval_status'] == 'Delayed']),
        len(interactions),
        len(interactions[interactions['issue_category'] == 'Late Payment'])
    ]
}

df_ops = pd.DataFrame(ops_summary)
df_ops.to_excel(writer, sheet_name='Operations Summary', index=False, startrow=3)

worksheet = writer.sheets['Operations Summary']
worksheet.write('A1', 'MONTHLY OPERATIONS REPORT', title_format)
worksheet.write('A2', f'Report Date: {datetime.now().strftime("%Y-%m-%d")}', subtitle_format)

for col_num, value in enumerate(df_ops.columns.values):
    worksheet.write(3, col_num, value, header_format)

worksheet.set_column('A:A', 25)
worksheet.set_column('B:B', 35)
worksheet.set_column('C:C', 25)

# -------------------------
# SHEET 2: Purchase Order Status
# -------------------------
po_summary = purchase_orders.groupby('approval_status').agg({
    'po_id': 'count',
    'po_amount': 'sum'
}).reset_index()
po_summary.columns = ['Approval Status', 'PO Count', 'Total Amount']
po_summary = po_summary.sort_values('PO Count', ascending=False)

po_summary.to_excel(writer, sheet_name='PO Status', index=False, startrow=2)

worksheet = writer.sheets['PO Status']
worksheet.write('A1', 'PURCHASE ORDER STATUS BREAKDOWN', title_format)

for col_num, value in enumerate(po_summary.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:A', 20)
worksheet.set_column('B:B', 15)
worksheet.set_column('C:C', 20)

# -------------------------
# SHEET 3: Customer Interactions
# -------------------------
interaction_summary = interactions.groupby('issue_category').agg({
    'interaction_id': 'count',
    'customer_id': 'nunique'
}).reset_index()
interaction_summary.columns = ['Issue Category', 'Total Interactions', 'Unique Customers']
interaction_summary = interaction_summary.sort_values('Total Interactions', ascending=False)

interaction_summary.to_excel(writer, sheet_name='Customer Interactions', index=False, startrow=2)

worksheet = writer.sheets['Customer Interactions']
worksheet.write('A1', 'CUSTOMER INTERACTION ANALYSIS', title_format)

for col_num, value in enumerate(interaction_summary.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:A', 25)
worksheet.set_column('B:C', 20)

# -------------------------
# SHEET 4: Payment Methods Analysis
# -------------------------
payment_methods = payments.groupby('payment_method').agg({
    'payment_id': 'count',
    'payment_amount': 'sum'
}).reset_index()
payment_methods.columns = ['Payment Method', 'Transaction Count', 'Total Amount']
payment_methods['% of Total'] = payment_methods['Total Amount'] / payment_methods['Total Amount'].sum()
payment_methods = payment_methods.sort_values('Total Amount', ascending=False)

payment_methods.to_excel(writer, sheet_name='Payment Methods', index=False, startrow=2)

worksheet = writer.sheets['Payment Methods']
worksheet.write('A1', 'PAYMENT METHODS BREAKDOWN', title_format)

for col_num, value in enumerate(payment_methods.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:A', 20)
worksheet.set_column('B:B', 20)
worksheet.set_column('C:C', 20)
worksheet.set_column('D:D', 15)

# -------------------------
# SHEET 5: Delayed PO Customers
# -------------------------
delayed_po_customers = purchase_orders[
    purchase_orders['approval_status'] == 'Delayed'
].merge(customers[['customer_id', 'customer_name', 'industry', 'city']], on='customer_id')

delayed_summary = delayed_po_customers.groupby(
    ['customer_id', 'customer_name', 'industry', 'city']
).agg({
    'po_id': 'count',
    'po_amount': 'sum'
}).reset_index()
delayed_summary.columns = ['Customer ID', 'Customer Name', 'Industry', 'City',
                           'Delayed PO Count', 'Total PO Amount']
delayed_summary = delayed_summary.sort_values('Delayed PO Count', ascending=False).head(20)

delayed_summary.to_excel(writer, sheet_name='Delayed PO Customers', index=False, startrow=2)

worksheet = writer.sheets['Delayed PO Customers']
worksheet.write('A1', 'TOP 20 CUSTOMERS WITH DELAYED POs', title_format)

for col_num, value in enumerate(delayed_summary.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:B', 15)
worksheet.set_column('C:D', 15)
worksheet.set_column('E:F', 18)

# -------------------------
# SHEET 6: Account Status Overview
# -------------------------
account_status = customers.groupby('account_status').agg({
    'customer_id': 'count'
}).reset_index()
account_status.columns = ['Account Status', 'Customer Count']

# Add AR info
status_ar = ar_master.groupby('account_status').agg({
    'outstanding_balance': 'sum'
}).reset_index()
status_ar.columns = ['Account Status', 'Total Outstanding AR']

account_status = account_status.merge(status_ar, on='Account Status', how='left')
account_status['Total Outstanding AR'] = account_status['Total Outstanding AR'].fillna(0)

account_status.to_excel(writer, sheet_name='Account Status', index=False, startrow=2)

worksheet = writer.sheets['Account Status']
worksheet.write('A1', 'CUSTOMER ACCOUNT STATUS', title_format)

for col_num, value in enumerate(account_status.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:A', 20)
worksheet.set_column('B:C', 25)

# -------------------------
# SHEET 7: Action Items
# -------------------------
# Customers with late payment interactions AND overdue AR
late_payment_customers = interactions[
    interactions['issue_category'] == 'Late Payment'
]['customer_id'].unique()

action_items = ar_master[
    (ar_master['customer_id'].isin(late_payment_customers)) &
    (ar_master['aging_bucket'].isin(['61-90', '90+']))
].groupby(['customer_id', 'customer_name', 'industry', 'city']).agg({
    'outstanding_balance': 'sum',
    'invoice_id': 'count',
    'days_past_due': 'max'
}).reset_index()

action_items.columns = ['Customer ID', 'Customer Name', 'Industry', 'City',
                       'Outstanding Balance', 'Overdue Invoices', 'Max Days Overdue']
action_items = action_items.sort_values('Outstanding Balance', ascending=False).head(30)

action_items.to_excel(writer, sheet_name='Priority Action Items', index=False, startrow=2)

worksheet = writer.sheets['Priority Action Items']
worksheet.write('A1', 'TOP 30 PRIORITY CUSTOMERS FOR FOLLOW-UP', title_format)

for col_num, value in enumerate(action_items.columns.values):
    worksheet.write(2, col_num, value, header_format)

worksheet.set_column('A:B', 15)
worksheet.set_column('C:D', 15)
worksheet.set_column('E:E', 20)
worksheet.set_column('F:G', 18)

# Save and close
writer.close()

print("✓ Monthly Operations Report generated: /content/Monthly_Operations_Report.xlsx")
print(f"  - Operations Summary")
print(f"  - PO Status")
print(f"  - Customer Interactions")
print(f"  - Payment Methods")
print(f"  - Delayed PO Customers")
print(f"  - Account Status")
print(f"  - Priority Action Items")

✓ Monthly Operations Report generated: /content/Monthly_Operations_Report.xlsx
  - Operations Summary
  - PO Status
  - Customer Interactions
  - Payment Methods
  - Delayed PO Customers
  - Account Status
  - Priority Action Items


**3**. **Decide what goes into the dashboard**

      a. How much AR do we have?

      b. How much is overdue?

      c. Where is the risk (aging)?

      d. Which customers need attention?

    1.1 Core KPIs (Top of Dashboard)

          a. Total AR

          b. Overdue AR

          c. % Overdue

          d. Average Days to Collect

    1.2 Core Charts

          a. AR by Aging Bucket

          b. Top 10 High-Risk Customers

          c. Payment Cycle Distribution

**4. Power BI–Ready Data Preparation (Python)**



* AR Master Table



This table contains:

    a. Invoice details

    b. Customer identifiers

    c. Outstanding balance

    d. Aging bucket

    e. Days past due

    f. Payment cycle days

In [65]:
# ar_master already contains all required calculated fields
ar_master.head()


,invoice_id,customer_id,invoice_date,invoice_amount,due_date,invoice_status,customer_name,industry,email,phone,city,account_status,payment_amount,outstanding_balance,days_past_due,aging_bucket,payment_date,payment_cycle_days
0,INV100000,C09769,2024-04-21,13110,2024-05-21,Open,Customer_9769,Healthcare,customer9769@company.com,6725814766,Washington DC,Active,13110.0,0.0,223,90+,2024-12-20,243.0
1,INV100001,C06097,2024-03-20,15641,2024-04-19,Open,Customer_6097,Logistics,customer6097@company.com,2452229400,Baltimore,Active,15641.0,0.0,255,90+,2024-09-30,194.0
2,INV100002,C07356,2024-02-17,20188,2024-03-18,Paid,Customer_7356,Retail,customer7356@company.com,6434990826,Rockville,Active,0.0,20188.0,287,90+,NaT,NaN
3,INV100003,C05507,2024-05-30,3609,2024-06-29,Paid,Customer_5507,Manufacturing,customer5507@company.com,7385363066,Baltimore,Active,3609.0,0.0,184,90+,2024-05-12,-18.0
4,INV100004,C08201,2024-03-31,5903,2024-04-30,Open,Customer_8201,Services,customer8201@company.com,3042810599,Washington DC,Active,5903.0,0.0,244,90+,2024-03-11,-20.0




*  Calculated Fields Included in Dataset



The following fields are pre-calculated in Python for consistency:

    a. days_past_due

    b. aging_bucket

    c. payment_cycle_days

    d. outstanding_balance

These fields enable:

    a. Aging analysis

    b. Risk segmentation

    c. Collection efficiency metrics



**3. Export Power BI Source Data**



In [66]:
ar_master.to_csv("/content/ar_master_powerbi.csv", index=False)
